In [1]:
%matplotlib qt

In [2]:
import sys
sys.path.append('/dls/science/groups/e02/Mohsen/code/Git_Repos/Merlin-Medipix/')
import epsic_tools.api as epsic

In [4]:
# # Saving the atomic fitting data
# import numpy as np
# import h5py
# import os
# def save_dict_to_hdf5(dic, filename):
#     """
#     ....
#     """
#     if os.path.exists(filename):
#         with h5py.File(filename, 'a') as h5file:
#             recursively_save_dict_contents_to_group(h5file, '/', dic)
#     else:
        
#         with h5py.File(filename, 'w') as h5file:
#             recursively_save_dict_contents_to_group(h5file, '/', dic)

# def recursively_save_dict_contents_to_group(h5file, path, dic):
#     """
#     ....
#     """
#     for key, item in dic.items():
#         if isinstance(item, (np.ndarray, list, float, int, str)):
#             h5file[path + key] = item
#         elif isinstance(item, dict):
#             recursively_save_dict_contents_to_group(h5file, path + key + '/', item)

            
# def load_dict_from_hdf5(filename):
#     """
#     ....
#     """
#     with h5py.File(filename, 'r') as h5file:
#         return recursively_load_dict_contents_from_group(h5file, '/')

# def recursively_load_dict_contents_from_group(h5file, path):
#     """
#     ....
#     """
#     ans = {}
#     for key, item in h5file[path].items():
#         if isinstance(item, h5py._hl.dataset.Dataset):
#             ans[key] = item[()]
#         elif isinstance(item, h5py._hl.group.Group):
#             ans[key] = recursively_load_dict_contents_from_group(h5file, path + key + '/')
#     return ans

In [5]:
data_file = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/fitting_results/Graphene_defect_25.0mrad_181.36A_def_2.17A_step_size.h5'
data_dict = load_dict_from_hdf5(data_file)

In [6]:
data_dict

{'atom_pos': array([[161.68821904, 164.40397512,   2.88087499,   3.45808138],
        [146.0519949 , 163.71566921,   1.46972364,   0.55848319],
        [ 72.83913155, 164.26900931,   1.3725945 ,   0.42495654],
        ...,
        [ 60.77515737,   3.37201579,   1.69148253,   2.02392036],
        [ 36.46536451,   3.57475785,   1.68410643,   1.95912481],
        [ 28.46505426,   3.64961524,   1.6439582 ,   2.00195949]]),
 'base_dir': '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_4June2020_512pixArray/Graphene_defect_25.0mrad_181.36A_def_2.17A_step_size',
 'experiment': {'data': {'data_path': '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_4June2020_512pixArray/Graphene_defect_25.0mrad_181.36A_def_2.17A_step_size/Graphene_defect_25.0mrad_181.36A_def_2.17A_step_size.h5',
   'dead_pixel_flag': 0,
   'flat_field_flag': 0,
   'key': '4DSTEM_simulation/data/datacubes/hdose_noisy_data',
   'load_flag': 1,
   'meta_type': 'hdf'}

In [7]:
data_dict.pop('atom_pos')

array([[161.68821904, 164.40397512,   2.88087499,   3.45808138],
       [146.0519949 , 163.71566921,   1.46972364,   0.55848319],
       [ 72.83913155, 164.26900931,   1.3725945 ,   0.42495654],
       ...,
       [ 60.77515737,   3.37201579,   1.69148253,   2.02392036],
       [ 36.46536451,   3.57475785,   1.68410643,   1.95912481],
       [ 28.46505426,   3.64961524,   1.6439582 ,   2.00195949]])

In [8]:
data_dict.pop('sampling_factor')

8.416889772148036

In [9]:
data_dict['json_path']

'/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_4June2020_512pixArray/Graphene_defect_25.0mrad_181.36A_def_2.17A_step_size/2000iter_20200604-155420.json'

In [10]:
import json
import os


def duplicate_json(source_json_path, new_json_path, param_to_change = None):
    """
    This gets a source json file and duplicates it, changing a parameter if declared
    Parameters
    ____________
    source_json_path: str
        full path of the starting json file
    new_json_path: str
        full path of the new json file
    param_to_change: tuple of (str, new_val) 
        default None - key of the parameter to be changed as first el of tuple and the new value as the second el.
    Returns
    ____________
    data_dict: dict
        The new dict if anything changed.
    
    TODO: Fix the ugly limited dict depth implementation!
    """
    data_dict = epsic.ptycho_utils.json_to_dict(source_json_path)
    if param_to_change == None:
        with open(new_json_path, 'w') as outfile:
            json.dump(data_dict, outfile, indent = 4)
    else:
        if isinstance(param_to_change[0], str):
            if _finditem(data_dict, param_to_change[0]) is not None:
                p = _finditem(data_dict, param_to_change[0])
                keys = _get_path(p)
#                 print(keys)
                keys.append(param_to_change[0])
                if len(keys)==4:
                    data_dict[keys[0]][keys[1]][keys[2]][keys[3]] = param_to_change[1]
                elif len(keys)==3:
                    data_dict[keys[0]][keys[1]][keys[2]] = param_to_change[1]
                elif len(keys)==2:
                    data_dict[keys[0]][keys[1]] = param_to_change[1]
                with open(new_json_path, 'w') as outfile:
                    json.dump(data_dict, outfile, indent = 4)
                return data_dict
            else:
                raise KeyError('The key provided does not exist in the dict.')                
        else:
            raise TypeError('param_to_change has to be a string.')
    return 


def _finditem(obj, key):
    if key in obj: return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = _finditem(v, key)
            path = []
            if item is not None:
                path.append('*'+k)
                return [item, path]
            
def _get_path(bad_list):
    path = []
    while len(bad_list) == 2:
        try:
            if bad_list[1][0][0] == '*':
                path.append(bad_list.pop()[0][1:])
                bad_list = bad_list[0]
                try: len(bad_list)
                except:
                    return path
        except: return path
    else:
        pass
    return path

# Defocus

In [11]:
_finditem(data_dict, 'defocus')

[[[array([1.81359e-08, 1.81359e-08]), ['*lens']], ['*optics']],
 ['*experiment']]

In [21]:
def_start_val = 1.81359e-08
pct_change = np.array([-20, -10, 10, 20])
def_new_vals = def_start_val * (1 + pct_change / 100)
print(def_new_vals)

[1.450872e-08 1.632231e-08 1.994949e-08 2.176308e-08]


In [18]:
save_path = '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_Defocus'
if os.path.exists(save_path) is False:
    os.mkdir(save_path)
    

In [36]:
for i in range(len(def_new_vals)):
    folder = os.path.join(save_path, 'defocus_'+str(np.round(def_new_vals[i] * 1e10,5))+'A')
    if os.path.exists(folder) is False:
        os.mkdir(folder)

    new_name = os.path.join(save_path, folder, 'ptyREX_defocus_'+str(np.round(def_new_vals[i] * 1e10,5))+'A.json')
    print(new_name)
    new_dict = duplicate_json(data_dict['json_path'], new_name, param_to_change=('defocus', [def_new_vals[i], def_new_vals[i]]))
    duplicate_json(new_name, new_name, param_to_change=('save_dir', os.path.dirname(new_name)))
    

/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_Defocus/defocus_145.0872A/ptyREX_defocus_145.0872A.json
/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_Defocus/defocus_163.2231A/ptyREX_defocus_163.2231A.json
/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_Defocus/defocus_199.4949A/ptyREX_defocus_199.4949A.json
/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_Defocus/defocus_217.6308A/ptyREX_defocus_217.6308A.json


In [37]:
run_dirs = epsic.sim_utils.get_ptyREX_ready(save_path)
print(run_dirs)

['/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_Defocus/defocus_163.2231A', '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_Defocus/defocus_199.4949A', '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_Defocus/defocus_217.6308A', '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_Defocus/defocus_145.0872A']


In [38]:
script_path = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix'
for path in run_dirs:
    it = os.scandir(path)
    for entry in it:
#         print(entry.name)
        if entry.is_file():
            if entry.name.startswith('ptyREX'):
                output_folder = os.path.dirname(entry.path)
                json_file = os.path.splitext(entry.name)[0]
#                 json_file = entry.name

                os.system('\n cd '+ output_folder + '\n module load global/cluster \n qsub '+ script_path + '/ptyREX_batch_submit.sh '+ output_folder + ' ' + json_file + ' 2000iter')

# Probe step size

In [43]:
_finditem(data_dict, 'dR')

[[[array([2.1737e-10, 2.1737e-10]), ['*scan']], ['*common']], ['*process']]

In [45]:
dR_start_val = 2.1737e-10
pct_change = np.array([-20, -10, 10, 20])
dR_new_vals = dR_start_val * (1 + pct_change / 100)
print(dR_new_vals)

[1.73896e-10 1.95633e-10 2.39107e-10 2.60844e-10]


In [46]:
save_path = '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_dR'
if os.path.exists(save_path) is False:
    os.mkdir(save_path)
    

In [47]:
for i in range(len(dR_new_vals)):
    folder = os.path.join(save_path, 'dR_'+str(np.round(dR_new_vals[i] * 1e10,5))+'A')
    if os.path.exists(folder) is False:
        os.mkdir(folder)

    new_name = os.path.join(save_path, folder, 'ptyREX_dR_'+str(np.round(def_new_vals[i] * 1e10,5))+'A.json')
    print(new_name)
    new_dict = duplicate_json(data_dict['json_path'], new_name, param_to_change=('dR', [dR_new_vals[i], dR_new_vals[i]]))
    duplicate_json(new_name, new_name, param_to_change=('save_dir', os.path.dirname(new_name)))
    

/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_dR/dR_1.73896A/ptyREX_dR_145.0872A.json
/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_dR/dR_1.95633A/ptyREX_dR_163.2231A.json
/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_dR/dR_2.39107A/ptyREX_dR_199.4949A.json
/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_dR/dR_2.60844A/ptyREX_dR_217.6308A.json


In [48]:
run_dirs = epsic.sim_utils.get_ptyREX_ready(save_path)
print(run_dirs)

['/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_dR/dR_1.73896A', '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_dR/dR_2.39107A', '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_dR/dR_1.95633A', '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_dR/dR_2.60844A']


In [49]:
script_path = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix'
for path in run_dirs:
    it = os.scandir(path)
    for entry in it:
#         print(entry.name)
        if entry.is_file():
            if entry.name.startswith('ptyREX'):
                output_folder = os.path.dirname(entry.path)
                json_file = os.path.splitext(entry.name)[0]
#                 json_file = entry.name

                os.system('\n cd '+ output_folder + '\n module load global/cluster \n qsub '+ script_path + '/ptyREX_batch_submit.sh '+ output_folder + ' ' + json_file + ' 2000iter')

# Scan rotation angle

In [162]:
save_path = '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot'
if os.path.exists(save_path) is False:
    os.mkdir(save_path)
    
rot_angles = np.arange(-10, 12, 2)
print(rot_angles)

[-10  -8  -6  -4  -2   0   2   4   6   8  10]


In [163]:
for i in range(len(rot_angles)):
    os.mkdir(os.path.join(save_path, 'rotation_'+str(rot_angles[i])))
    folder = os.path.join(save_path, 'rotation_'+str(rot_angles[i]))
    new_name = os.path.join(save_path, folder, 'ptyREX_rotation_'+str(rot_angles[i])+'.json')
    new_dict = duplicate_json(data_dict['json_path'], new_name, param_to_change=('rotation', int(rot_angles[i])))
    duplicate_json(new_name, new_name, param_to_change=('save_dir', os.path.dirname(new_name)))
    
    

In [164]:
run_dirs = epsic.sim_utils.get_ptyREX_ready(save_path)

In [165]:
run_dirs

['/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_4',
 '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_-6',
 '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_8',
 '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_-8',
 '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_2',
 '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_0',
 '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_10',
 '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_-10',
 '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_-2',
 '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_-4',
 '/dls/e02/data/2020/cm

In [178]:
script_path = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix'
for path in run_dirs:
    it = os.scandir(path)
    for entry in it:
#         print(entry.name)
        if entry.is_file():
            if entry.name.startswith('ptyREX'):
                output_folder = os.path.dirname(entry.path)
                json_file = os.path.splitext(entry.name)[0]
#                 json_file = entry.name

                os.system('\n cd '+ output_folder + '\n module load global/cluster \n qsub '+ script_path + '/ptyREX_batch_submit.sh '+ output_folder + ' ' + json_file + ' 2000iter')

In [8]:
epsic.ptycho_utils.plot_ptyREX_output('/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptycREX_ScanRot/rotation_-10/2000iter_20200607-233637.json')

In [39]:
data_dict


{'base_dir': '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_4June2020_512pixArray/Graphene_defect_25.0mrad_181.36A_def_2.17A_step_size',
 'experiment': {'data': {'data_path': '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_4June2020_512pixArray/Graphene_defect_25.0mrad_181.36A_def_2.17A_step_size/Graphene_defect_25.0mrad_181.36A_def_2.17A_step_size.h5',
   'dead_pixel_flag': 0,
   'flat_field_flag': 0,
   'key': '4DSTEM_simulation/data/datacubes/hdose_noisy_data',
   'load_flag': 1,
   'meta_type': 'hdf'},
  'detector': {'position': array([0.        , 0.        , 0.11470963])},
  'optics': {'FZP': {'use': 0},
   'diffuser': {'use': 0},
   'lens': {'alpha': 0.05,
    'defocus': array([1.81359e-08, 1.81359e-08]),
    'use': 1},
   'pinhole': {'use': 0}},
  'sample': {'position': array([0, 0, 0])}},
 'json_path': '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_4June2020_512pixArray/Graphene

In [5]:
epsic.ptycho_utils.plot_ptyREX_output('/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_ptyREX_dR/dR_2.60844A/2000iter_20200610-184757.json')

In [6]:
correct_cond = '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_4June2020_512pixArray/Graphene_defect_25.0mrad_181.36A_def_2.17A_step_size/2000iter_20200604-155420.json'
epsic.ptycho_utils.plot_ptyREX_output(correct_cond)